In [1]:
import json
import time
import subprocess
import math
import regex
import sys
import os

import spacy
from tqdm import tqdm
from nltk.corpus import wordnet as wn
import numpy as np
from sklearn.model_selection import KFold

from json_parser import Parser
from common import BBT_PEOPLE, BoundingBox
from image_processing import ObjectDetector
import language_processing as lp
import utils
from utils import split_data_set
import inference as inf

In [2]:
with open('hold_base.lp') as f:
    background_knowledge = f.read().splitlines()

parser = Parser()

# With Ground Truth Bounding Box

In [3]:
def one_iter_gt(train_set, test_set):
    # 3. Test on test set
    jacc_score = 0
    num_questions = 0
    parsing_error = []
    not_full_score = []
    zero_score = []

    for test in test_set:
        pred_ans_idx = inf.inference(test, gt_object=True)
        if pred_ans_idx == [-1]:
            parsing_error.append(test['qid'])
            continue

        score = inf.get_jaccard_score(pred_ans_idx, test)

        if 0 < score < 1:
            not_full_score.append(test['qid'])
        elif score == 0:
            zero_score.append(test['qid'])

        jacc_score += score
        num_questions += 1

    ##########################################################
    
    # 4. Print out result
    print(F'# tests:     {num_questions}')
    print(F'jacc score:   {jacc_score}')
    print(F'norm jacc score: {jacc_score / num_questions}')
    print()

    print('PARSING ERROR')
    print(parsing_error)
    print()

    print('NOT FULL SCORE')
    print(not_full_score)
    print()

    print('ZERO SCORE')
    print(zero_score)
    
    return jacc_score, num_questions

In [4]:
with open('hold_questions.json') as f:
    attempted_question = json.load(f)

print(F'# total tests: {len(attempted_question)}')

a_q_dict = dict()

for q in attempted_question:
    qid = q['qid']
    a_q_dict[qid] = q
    
kf = KFold(n_splits=5)
kf.get_n_splits(attempted_question)

splits = kf.split(attempted_question)

run_id = 0
total_jacc_score = 0
total_questions = 0

# total tests: 886


# Run 1

In [5]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_gt', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1675


In [6]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     177
jacc score:   125.03333333333333
norm jacc score: 0.7064030131826742

PARSING ERROR
[91833]

NOT FULL SCORE
[85689, 72563, 52058, 87060, 94890, 78852, 71186, 33374, 51101, 63329, 12193, 61212, 3120]

ZERO SCORE
[59566, 45810, 113521, 107346, 117818, 1260, 18350, 65891, 9143, 60940, 63936, 50348, 38557, 36135, 90819, 13001, 52351, 83808, 48275, 21145, 50385, 121282, 102623, 119619, 45155, 93719, 20082, 76092, 25385, 34678, 112266, 94974, 67705, 45415, 62681, 33258, 1210, 66632, 15575, 41227, 71942, 108251, 37191, 2540, 50755]


# Run 2

In [7]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_gt', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1752


In [8]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   128.43333333333334
norm jacc score: 0.733904761904762

PARSING ERROR
[110880, 11144]

NOT FULL SCORE
[81523, 62904, 45051, 92313, 101817, 68703, 6457, 16543, 81406, 95158, 80467]

ZERO SCORE
[38543, 117851, 60166, 3969, 11773, 46461, 118476, 52597, 87330, 16559, 63962, 55624, 94366, 104234, 41489, 83645, 107768, 3888, 12408, 86452, 26737, 45011, 11818, 17278, 66631, 79944, 114436, 66736, 19447, 16182, 48221, 19331, 68289, 20562, 111835, 84532, 38737, 28204, 22658, 8575]


# Run 3

In [9]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_gt', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1751


In [10]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     177
jacc score:   124.16666666666667
norm jacc score: 0.7015065913370998

PARSING ERROR
[]

NOT FULL SCORE
[17424, 97636, 4797, 97719, 122032, 107372, 33418, 28253, 120918, 110401, 6478, 13720, 67921, 75310, 47217]

ZERO SCORE
[32606, 1842, 28739, 118198, 113145, 12737, 32579, 73159, 79282, 26776, 9808, 97405, 45895, 12822, 60959, 49133, 57218, 119983, 34666, 26986, 7558, 16146, 50524, 79329, 30878, 36166, 27617, 20340, 38381, 48743, 18042, 84903, 42612, 90782, 22192, 66214, 75040, 89083, 34699, 107151, 58701, 81589, 6553, 63602, 115376]


# Run 4

In [11]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_gt', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1722


In [13]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   124.94999999999999
norm jacc score: 0.714

PARSING ERROR
[5374, 64394]

NOT FULL SCORE
[94232, 100045, 101233, 119541, 81602, 34859, 47714, 115405, 38768, 62756, 78301, 92516, 10847, 73907, 2179, 101976]

ZERO SCORE
[54981, 58600, 108741, 48878, 60504, 58347, 42604, 98014, 111420, 41401, 104181, 112558, 623, 90459, 96813, 43717, 35102, 13796, 7398, 16523, 61614, 100757, 115726, 64917, 28343, 10881, 22928, 29251, 1112, 29320, 47096, 4770, 10199, 57838, 91217, 90367, 60429, 111187, 102598, 89700, 16643]


# Run 5

In [14]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_gt', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1760


In [15]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   125.83333333333333
norm jacc score: 0.719047619047619

PARSING ERROR
[56128, 116486]

NOT FULL SCORE
[41402, 90225, 117747, 109516]

ZERO SCORE
[44553, 40734, 54160, 113117, 33554, 105758, 17298, 14773, 93301, 56096, 112862, 37031, 103778, 104027, 96969, 71850, 7962, 83484, 27275, 14598, 95907, 94077, 42450, 677, 6753, 63786, 57942, 276, 25490, 42970, 84175, 14167, 45751, 87754, 114725, 40491, 106967, 32721, 45905, 80819, 19711, 13911, 117755, 121820, 27411, 60915, 114775]


In [17]:
print(F'Total jacc score: {total_jacc_score}')
print(F'Total questions:  {total_questions}')
print(F'Avg norm jacc:    {total_jacc_score /  total_questions}')


Total jacc score: 628.4166666666666
Total questions:  879
Avg norm jacc:    0.7149222601441031


In [ ]:
from image_processing import draw_bounding_box
from PIL import Image
from IPython.display import display


test = a_q_dict[25720]

# time_span_to_timestamps_list(test)
vid_folder = frame_folder + test['vid_name'] + '/'

time = 30
qa_objects = od.get_frame_qa_objects(vid_folder, 0.7, time)
bboxes = [o.bbox for o in qa_objects]
display(Image.fromarray(draw_bounding_box(vid_folder + F'000{time}.jpg', bboxes)))